# 8. Загрузить данные в spark

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [52]:
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz

spark-2.4.4-bin-hadoop2.7/
spark-2.4.4-bin-hadoop2.7/R/
spark-2.4.4-bin-hadoop2.7/R/lib/
spark-2.4.4-bin-hadoop2.7/R/lib/sparkr.zip
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/R.css
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/00Index.html
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/aliases.rds
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/AnIndex
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/SparkR.rdx
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/SparkR.rdb
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/paths.rds
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/worker.R
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/daemon.R
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/testthat/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/testthat/te

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

Имена столбцов задала еще в csv файле. Так мне показалось быстрее

In [0]:
movies = spark.read.csv('item.csv', sep = '|', inferSchema=True, header=True)

In [57]:
movies.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_ID|         movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|

In [58]:
movies.printSchema()

root
 |-- movie_ID: integer (nullable = true)
 |-- movie title: string (nullable = true)
 |-- release date: string (nullable = true)
 |-- video release date: string (nullable = true)
 |-- IMDb URL: string (nullable = true)
 |-- unknown: integer (nullable = true)
 |-- Action: integer (nullable = true)
 |-- Adventure: integer (nullable = true)
 |-- Animation: integer (nullable = true)
 |-- Children's: integer (nullable = true)
 |-- Comedy: integer (nullable = true)
 |-- Crime: integer (nullable = true)
 |-- Documentary: integer (nullable = true)
 |-- Drama: integer (nullable = true)
 |-- Fantasy: integer (nullable = true)
 |-- Film-Noir: integer (nullable = true)
 |-- Horror: integer (nullable = true)
 |-- Musical: integer (nullable = true)
 |-- Mystery: integer (nullable = true)
 |-- Romance: integer (nullable = true)
 |-- Sci-Fi: integer (nullable = true)
 |-- Thriller: integer (nullable = true)
 |-- War: integer (nullable = true)
 |-- Western: integer (nullable = true)



In [0]:
ratings = spark.read.csv('data.csv', sep = '\t', inferSchema=True, header=True)

In [60]:
ratings.show()

+-------+--------+------+---------+
|user_ID|movie_ID|Rating|Timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



In [61]:
ratings.printSchema()

root
 |-- user_ID: integer (nullable = true)
 |-- movie_ID: integer (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Timestamp: integer (nullable = true)



# 9. Средствами спарка вывести среднюю оценку для каждого фильма

In [0]:
from pyspark.sql.functions import format_number

In [63]:
ratings.select(['movie_ID','Rating']).groupby("movie_ID").mean('Rating').show()

+--------+------------------+
|movie_ID|       avg(Rating)|
+--------+------------------+
|     496| 4.121212121212121|
|     471|3.6108597285067874|
|     463| 3.859154929577465|
|     148|          3.203125|
|    1342|               2.5|
|     833| 3.204081632653061|
|    1088| 2.230769230769231|
|    1591|3.1666666666666665|
|    1238|             3.125|
|    1580|               1.0|
|    1645|               4.0|
|     392|3.5441176470588234|
|     623| 2.923076923076923|
|     540| 2.511627906976744|
|     858|               1.0|
|     737| 2.983050847457627|
|     243|2.4393939393939394|
|    1025|2.9318181818181817|
|    1084| 3.857142857142857|
|    1127| 2.909090909090909|
+--------+------------------+
only showing top 20 rows



# 10. Посчитайте средствами спарка среднюю оценку для каждого жанра

In [64]:
movies_dropped = movies.select(['movie_ID', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
movies_dropped.show()

+--------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_ID|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       1|  0|      0|
|       3|      0|     0|        0|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       1|  0|      0|
|       4|

In [0]:
ratings_dropped = ratings.select(['movie_ID', 'Rating'])

In [72]:
from pyspark.sql import Row
movies_with_avg= ratings_dropped.join(movies_dropped, 'movie_ID', how = 'inner')
movies_with_avg.show()

+--------+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_ID|Rating|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     242|     3|      0|     0|        0|        0|         0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     302|     3|      0|     0|        0|        0|         0|     0|    1|          0|    0|      0|        1|     0|      0|      1|      0|     0|       1|  0|      0|
|     377|     1|      0|     0|        0|        0|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|     

Не самое изящное решение

In [73]:
movies_with_avg_unknown = movies_with_avg.where(movies_with_avg['unknown'] == 1).select(['unknown','Rating']).groupby("unknown").mean('Rating').show()

+-------+-----------+
|unknown|avg(Rating)|
+-------+-----------+
|      1|        3.2|
+-------+-----------+



In [85]:
movies_with_avg_Action = movies_with_avg.where(movies_with_avg['Action'] == 1).select(['Action','Rating']).groupby("Action").mean('Rating').show()

+------+-----------------+
|Action|      avg(Rating)|
+------+-----------------+
|     1|3.480245417953027|
+------+-----------------+



In [91]:
movies_with_avg_Adventure = movies_with_avg.where(movies_with_avg['Adventure'] == 1).select(['Adventure','Rating']).groupby("Adventure").mean('Rating').show()

+---------+-----------------+
|Adventure|      avg(Rating)|
+---------+-----------------+
|        1|3.503526503308369|
+---------+-----------------+



In [92]:
movies_with_avg_Animation = movies_with_avg.where(movies_with_avg['Animation'] == 1).select(['Animation','Rating']).groupby("Animation").mean('Rating').show()

+---------+------------------+
|Animation|       avg(Rating)|
+---------+------------------+
|        1|3.5766990291262135|
+---------+------------------+



In [93]:
movies_with_avg_Children = movies_with_avg.where(movies_with_avg["Children's"] == 1).select(["Children's",'Rating']).groupby("Children's").mean('Rating').show()

+----------+------------------+
|Children's|       avg(Rating)|
+----------+------------------+
|         1|3.3532442216652742|
+----------+------------------+



In [94]:
movies_with_avg_Comedy = movies_with_avg.where(movies_with_avg["Comedy"] == 1).select(["Comedy",'Rating']).groupby("Comedy").mean('Rating').show()

+------+------------------+
|Comedy|       avg(Rating)|
+------+------------------+
|     1|3.3940734781442745|
+------+------------------+



In [95]:
movies_with_avg_Crime = movies_with_avg.where(movies_with_avg["Crime"] == 1).select(["Crime",'Rating']).groupby("Crime").mean('Rating').show()

+-----+------------------+
|Crime|       avg(Rating)|
+-----+------------------+
|    1|3.6322780881440098|
+-----+------------------+



In [96]:
movies_with_avg_Documentary = movies_with_avg.where(movies_with_avg["Documentary"] == 1).select(["Documentary",'Rating']).groupby("Documentary").mean('Rating').show()

+-----------+------------------+
|Documentary|       avg(Rating)|
+-----------+------------------+
|          1|3.6728232189973613|
+-----------+------------------+



In [97]:
movies_with_avg_Drama = movies_with_avg.where(movies_with_avg["Drama"] == 1).select(["Drama",'Rating']).groupby("Drama").mean('Rating').show()

+-----+------------------+
|Drama|       avg(Rating)|
+-----+------------------+
|    1|3.6873793708484772|
+-----+------------------+



In [98]:
movies_with_avg_Fantasy = movies_with_avg.where(movies_with_avg["Fantasy"] == 1).select(["Fantasy",'Rating']).groupby("Fantasy").mean('Rating').show()

+-------+------------------+
|Fantasy|       avg(Rating)|
+-------+------------------+
|      1|3.2152366863905324|
+-------+------------------+



In [100]:
movies_with_avg_Film_Noir = movies_with_avg.where(movies_with_avg["Film-Noir"] == 1).select(["Film-Noir",'Rating']).groupby("Film-Noir").mean('Rating').show()

+---------+------------------+
|Film-Noir|       avg(Rating)|
+---------+------------------+
|        1|3.9215233698788228|
+---------+------------------+



In [101]:
movies_with_avg_Horror = movies_with_avg.where(movies_with_avg["Horror"] == 1).select(["Horror",'Rating']).groupby("Horror").mean('Rating').show()

+------+------------------+
|Horror|       avg(Rating)|
+------+------------------+
|     1|3.2903893172841827|
+------+------------------+



In [102]:
movies_with_avg_Musical = movies_with_avg.where(movies_with_avg["Musical"] == 1).select(["Musical",'Rating']).groupby("Musical").mean('Rating').show()

+-------+-----------------+
|Musical|      avg(Rating)|
+-------+-----------------+
|      1|3.521396851029471|
+-------+-----------------+



In [103]:
movies_with_avg_Mystery = movies_with_avg.where(movies_with_avg["Mystery"] == 1).select(["Mystery",'Rating']).groupby("Mystery").mean('Rating').show()

+-------+----------------+
|Mystery|     avg(Rating)|
+-------+----------------+
|      1|3.63813155386082|
+-------+----------------+



In [104]:
movies_with_avg_Romance = movies_with_avg.where(movies_with_avg["Romance"] == 1).select(["Romance",'Rating']).groupby("Romance").mean('Rating').show()

+-------+-----------------+
|Romance|      avg(Rating)|
+-------+-----------------+
|      1|3.621704948358255|
+-------+-----------------+



In [105]:
movies_with_avg_Sci_Fi = movies_with_avg.where(movies_with_avg["Sci-Fi"] == 1).select(["Sci-Fi",'Rating']).groupby("Sci-Fi").mean('Rating').show()

+------+------------------+
|Sci-Fi|       avg(Rating)|
+------+------------------+
|     1|3.5607227022780834|
+------+------------------+



In [106]:
movies_with_avg_Thriller = movies_with_avg.where(movies_with_avg["Thriller"] == 1).select(["Thriller",'Rating']).groupby("Thriller").mean('Rating').show()

+--------+------------------+
|Thriller|       avg(Rating)|
+--------+------------------+
|       1|3.5090069495245064|
+--------+------------------+



In [107]:
movies_with_avg_War = movies_with_avg.where(movies_with_avg["War"] == 1).select(["War",'Rating']).groupby("War").mean('Rating').show()

+---+-----------------+
|War|      avg(Rating)|
+---+-----------------+
|  1|3.815811874866993|
+---+-----------------+



In [108]:
movies_with_avg_Western = movies_with_avg.where(movies_with_avg["Western"] == 1).select(["Western",'Rating']).groupby("Western").mean('Rating').show()

+-------+------------------+
|Western|       avg(Rating)|
+-------+------------------+
|      1|3.6132686084142396|
+-------+------------------+



# 11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)

Сначала создаю датафреймы по количеству оценок

In [0]:
from pyspark.sql.functions import col
top5 = movies_with_avg.select(["movie_ID",'Rating']).groupby("movie_ID").count().sort(col("count").desc()).limit(5)

In [147]:
top5.show()

+--------+-----+
|movie_ID|count|
+--------+-----+
|      50|  583|
|     258|  509|
|     100|  508|
|     181|  507|
|     294|  485|
+--------+-----+



In [0]:
down5 = movies_with_avg.select(["movie_ID",'Rating']).groupby("movie_ID").count().sort("count").limit(5)


In [148]:
down5.show()

+--------+-----+
|movie_ID|count|
+--------+-----+
|    1618|    1|
|    1533|    1|
|    1339|    1|
|    1460|    1|
|    1650|    1|
+--------+-----+



Датафреймы по самой оценке

In [152]:
top5_score = movies_with_avg.select(["movie_ID",'Rating']).sort(col("Rating").desc()).limit(5).show()

+--------+------+
|movie_ID|Rating|
+--------+------+
|      95|     5|
|     201|     5|
|     234|     5|
|     465|     5|
|     603|     5|
+--------+------+



In [153]:
down5_score = movies_with_avg.select(["movie_ID",'Rating']).sort(col("Rating").asc()).limit(5).show()

+--------+------+
|movie_ID|Rating|
+--------+------+
|    1081|     1|
|    1049|     1|
|     796|     1|
|     346|     1|
|     288|     1|
+--------+------+

